In [ ]:
#Importação da bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

url = "https://raw.githubusercontent.com/amandacmelo/Projeto-Ciencia-de-Dados/main/dados_limpos.csv"
dados = pd.read_csv(url, sep=",")
# Substitui valores 0 por NaN em PIB_PerCapita (para evitar distorções nas análises)
dados['PIB_PerCapita'] = dados['PIB_PerCapita'].replace(0, np.nan)


warnings.filterwarnings("ignore")

#dados = pd.read_csv(url, on_bad_lines="skip", sep=";") #lida com linhas com algum espaço vazio
dados.columns

Index(['Ano', 'CDMunicipio', 'Populacao', 'PessoalOcupado',
       'PessoalAssalariado', 'VrSalarios', 'PIB', 'QtEmpresas',
       'AreaPlantada_h', 'AreaColhida_h', 'VlProducaoAgricola',
       'VlProducaoPecuaria', 'Area', 'Importacoes_US$', 'Exportacoes_US$',
       'Receitas_R$', 'Transferencias_correntes_R$',
       'Transferencias_capital_R$', 'NrNascimentos', 'NrObitosInfantis',
       'povoamento', 'UF', 'Municipio', 'Regiao', 'PIB_PerCapita',
       'Densidade_Populacional', 'Proporcao_PessoalAssalariado',
       'Proporcao_Area_Plantada'],
      dtype='object')

# 🟠 **Inferência Estatística e Regras de Associação - Economia e Empregabilidade**

Nesta etapa, a equipe deu continuidade à análise de dados por meio da aplicação de técnicas de inferência estatística e algoritmos de regras de associação. Este documento tem como objetivo aprofundar a investigação dos aspectos relacionados à economia e à empregabilidade, com o intuito de avançar na busca por respostas às perguntas de pesquisa. Para isso, foram realizados testes para avaliar a significância dos padrões observados e, paralelamente, utilizou-se um algoritmo de regras de associação para extrair combinações de variáveis com maior frequência de ocorrência conjunta nos dados.


## 🔍 Inferência Estatística

Durante a análise exploratória dos dados, surgiram novas indagações. Diante disso, o grupo optou por aprofundar a investigação da relação entre o PIB per capita e a proporção de pessoal assalariado nos municípios. O objetivo é verificar se há uma diferença significativa no PIB per capita médio entre municípios com alta e baixa proporção de pessoal assalariado.
A partir dessa análise, surge a seguinte pergunta:
**"Municípios com maior proporção de pessoal assalariado apresentam um PIB per capita significativamente diferente dos demais?"**

Para iniciar o processo de resposta a essa questão, será aplicada uma inferência estatística por meio de um teste de hipótese, permitindo uma avaliação rigorosa da existência ou não dessa diferença entre os grupos analisados.

Nesse contexto, foram elaboradas duas hipóteses para a realização da análise entre dois grupos independentes:

- **Hipótese nula (H₀):** Não há diferença significativa no PIB per capita médio entre os dois grupos de municípios.
- **Hipótese alternativa (H₁):** Existe diferença significativa no PIB per capita médio entre os dois grupos.







###🔹 Escolha da amostra

Os municípios serão divididos em dois grupos:
- **Alta proporção de pessoal assalariado:** Municípios com proporção de pessoal assalariado maior ou igual ao terceiro quartil (Q3, 25% superiores).

- **Baixa proporção de pessoal assalariado:** Municípios com proporção de pessoal assalariado menor ou igual ao primeiro quartil (Q1, 25% inferiores).

Para separar os municípios em grupos distintos, utilizou-se a divisão por quartis da variável **Proporcao_PessoalAssalariado**:

Este critério permite focar nos extremos da variável, facilitando a detecção de municípios mais diferentes entre si, reduz o ruído ao eliminar o grupo intermediário, onde as diferenças podem ser mais sutis e por fim evita distorções pois é menos sensível a outliers.


In [ ]:
q1 = dados['Proporcao_PessoalAssalariado'].quantile(0.25)
q3 = dados['Proporcao_PessoalAssalariado'].quantile(0.75)

grupo_baixo = dados[dados['Proporcao_PessoalAssalariado'] <= q1]
grupo_alto = dados[dados['Proporcao_PessoalAssalariado'] >= q3]


### 🔹 Estatítisca Descritiva da Amostra

Explorar as características principais do PIB per capita nos dois grupos definidos (alto e baixo pessoal assalariado), calculando:

- Tamanho do grupo  
- Média  
- Desvio padrão  
- Erro padrão da média  
- Distância entre as médias dos grupos em unidades do erro padrão combinado  

Essa análise auxilia na compreensão das diferenças entre os grupos antes do teste de hipótese.



In [ ]:
import numpy as np

def estatisticas_comparacao(grupo1, grupo2, nome1, nome2):
    n1 = grupo1.size
    media1 = grupo1.mean()
    desvio1 = grupo1.std()
    erro_padrao1 = desvio1 / np.sqrt(n1)

    n2 = grupo2.size
    media2 = grupo2.mean()
    desvio2 = grupo2.std()
    erro_padrao2 = desvio2 / np.sqrt(n2)

    erro_padrao_comb = np.sqrt(erro_padrao1**2 + erro_padrao2**2)

    distancia = abs(media1 - media2) / erro_padrao_comb

    print(f"Estatísticas para {nome1}:")
    print(f"  Tamanho: {n1}")
    print(f"  Média PIB per capita: {media1:.2f}")
    print(f"  Desvio padrão: {desvio1:.2f}")
    print(f"  Erro padrão da média: {erro_padrao1:.2f}\n")

    print(f"Estatísticas para {nome2}:")
    print(f"  Tamanho: {n2}")
    print(f"  Média PIB per capita: {media2:.2f}")
    print(f"  Desvio padrão: {desvio2:.2f}")
    print(f"  Erro padrão da média: {erro_padrao2:.2f}\n")

    print(f"Distância entre as médias (em erros padrão combinados): {distancia:.2f}")

grupo_baixo_pib = grupo_baixo['PIB_PerCapita'].dropna()
grupo_alto_pib = grupo_alto['PIB_PerCapita'].dropna()

estatisticas_comparacao(grupo_baixo_pib, grupo_alto_pib, "Grupo Baixo", "Grupo Alto")


Estatísticas para Grupo Baixo:
  Tamanho: 7032
  Média PIB per capita: 27.02
  Desvio padrão: 20.16
  Erro padrão da média: 0.24

Estatísticas para Grupo Alto:
  Tamanho: 6873
  Média PIB per capita: 19.52
  Desvio padrão: 30.73
  Erro padrão da média: 0.37

Distância entre as médias (em erros padrão combinados): 16.97


| Estatística                 | Grupo Baixo            | Grupo Alto            |
|----------------------------|-----------------------|----------------------|
| Tamanho                    | 7032                 | 6873              |
| Média PIB per capita        | 27,02                | 19,52               |
| Desvio padrão              | 20,16                | 30,73                |
| Erro padrão da média       | 0,24                 | 0,37                |

**Distância entre as médias (em erros padrão combinados): 16,97**

Os dados indicam que há uma diferença significativa no PIB per capita entre os municípios com baixa e alta proporção de pessoal assalariado. O grupo com baixa proporção apresenta uma média de PIB per capita maior (27,02) em comparação ao grupo com alta proporção (19,52), com uma distância entre as médias muito elevada (16,97 erros padrão combinados), indicando que essa diferença não é fruto do acaso, mas sim estatisticamente robusta. Além disso, a baixa variabilidade do erro padrão devido ao grande tamanho das amostras reforça a precisão dessas estimativas, sugerindo que a proporção de pessoal assalariado está associada a diferenças relevantes na renda média dos municípios.


### 🔹 Teste t de Welch para Diferença entre Médias do PIB Per Capita

O teste t de Welcj é aplicado para comparar as médias do PIB per capita entre os dois grupos, considerando que as variâncias podem ser diferentes.

Calcula-se a estatística t e o valor-p (p-valor).

**Critério de decisão:**

- Se \( p < 0,05 \), rejeita-se a hipótese nula, ou seja, há diferença significativa entre as médias dos grupos.
- Se \( p >= 0,05 \), não se rejeita a hipótese nula, indicando que não há evidência suficiente para afirmar que as médias são diferentes.



In [ ]:
from scipy.stats import ttest_ind

grupo_baixo_pib = grupo_baixo['PIB_PerCapita'].dropna()
grupo_alto_pib = grupo_alto['PIB_PerCapita'].dropna()

stat, pvalue = ttest_ind(grupo_baixo_pib, grupo_alto_pib, equal_var=False)

print("### Teste t de Welch - Comparação entre grupos")
print(f"Estatística t: {stat:.2f}")
print(f"Valor-p: {pvalue:}")

alpha = 0.05
if pvalue < alpha:
    print("Resultado: Rejeita-se a hipótese nula. Há evidência de diferença significativa entre as médias.")
else:
    print("Resultado: Não se rejeita a hipótese nula. Não há evidência de diferença significativa entre as médias.")


### Teste t de Welch - Comparação entre grupos
Estatística t: 16.97
Valor-p: 7.197223193425474e-64
Resultado: Rejeita-se a hipótese nula. Há evidência de diferença significativa entre as médias.


### 🔹 Conclusão

Os resultados do teste t de Welch indicam uma **diferença estatisticamente significativa** entre as médias de PIB per capita dos municípios com alta e baixa proporção de pessoal assalariado. Com um valor-p extremamente baixo (≈ 7.2 × 10⁻⁶⁴), **rejeita-se a hipótese nula**, que afirmava não haver diferença entre os grupos. Assim, os dados sugerem fortemente que a proporção de pessoal assalariado está associada a variações significativas no PIB per capita dos municípios analisados.
Além disso, com base na pergunta levantada e com os resultados obtidos é possível afirmar que municípios com maior proporção de pessoal assalariado apresentam um PIB per capita significativamente diferente dos demais.Contudo, vale destacar: o teste mostra diferença significativa, mas não afirma causalidade — ou seja, não se pode dizer com certeza que mais pessoal assalariado causa maior (ou menor) PIB per capita, apenas que existe uma associação entre esses fatores nos dados analisados.
